# Volunteer model loss calculation

This notebook takes a `subject_id` and calculates the MSE of each volunteer's model to the galaxy data. These losses are then glued to the notebook for use elsewhere.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from gzbuilder_analysis import parsing, rendering, fitting
import lib.galaxy_utilities as gu
import scrapbook as sb

/opt/apps/pkgs/anaconda3/2019.03/intel64/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [3]:
subject_id = 20902040

In [4]:
# Parameters
subject_id = 21096940


In [5]:
diff_data_df = pd.read_pickle('lib/fitting_metadata.pkl')

In [6]:
def get_losses_by_user(subject_id):
  diff_data = diff_data_df.loc[subject_id]
  psf = diff_data['psf']
  pixel_mask = np.array(diff_data['pixel_mask'])[::-1]
  galaxy_data = np.array(diff_data['galaxy_data'])[::-1]
  image_size = galaxy_data.shape[0]
  size_diff = diff_data['size_diff']
  
  cls = gu.classifications.query('subject_ids == {}'.format(subject_id))
  names = cls.user_name 
  annotations = cls['annotations'].apply(json.loads)

  models = annotations.apply(
    parsing.parse_annotation,
    size_diff=size_diff
  )

  tqdm.pandas(
    desc='Rendering models'.format(subject_id),
    leave=False
  )
  rendered = models.progress_apply(
    rendering.calculate_model,
    image_size=image_size,
    psf=psf
  ).rename('rendered')

  tqdm.pandas(
    desc='Calculating differences'.format(subject_id),
    leave=False
  )
  losses = rendered.apply(
    fitting.loss,
    args=(galaxy_data, pixel_mask)
  ).rename('loss')
  all_losses = pd.Series(
    losses.values,
    index=names.values
  ).rename(subject_id)
  best_losses = pd.concat((
    all_losses.where(
      ~all_losses.duplicated(keep=False)
    ).dropna(), # all non-duplicates
    all_losses.where(
      all_losses.duplicated(keep=False)
    ).dropna().groupby(level=0).min() # best value of duplicates
  ), axis=0)
  return best_losses

In [7]:
losses = get_losses_by_user(subject_id)
losses

Rendering models:   0%|          | 0/30 [00:00<?, ?it/s]

Rendering models:   7%|▋         | 2/30 [00:04<01:00,  2.16s/it]

Rendering models:  10%|█         | 3/30 [00:06<00:57,  2.14s/it]

Rendering models:  13%|█▎        | 4/30 [00:07<00:48,  1.87s/it]

Rendering models:  20%|██        | 6/30 [00:08<00:32,  1.37s/it]

...

Rendering models:  87%|████████▋ | 26/30 [00:15<00:01,  2.43it/s]

Rendering models:  93%|█████████▎| 28/30 [00:15<00:00,  3.22it/s]

Rendering models:  97%|█████████▋| 29/30 [00:15<00:00,  3.93it/s]

Rendering models: 100%|██████████| 30/30 [00:15<00:00,  3.99it/s]

not-logged-in-3b0b929f4475fd276561    0.041110
Jorgearcelarreta                      0.000946
jnarayanbvg                           0.002724
awright5                              0.001904
TSM_Ginga                             0.001618
salmi.a01                             0.001752
Hunting_X                             0.001094
not-logged-in-a722508d1bb33173b3c2    0.003552
redboy21                              0.068539
sn346808                              0.001237
Daniel.brooker2                       0.001184
LeeJoseph                             0.013420
ElisabethB                            0.040085
Tenorclef                             0.001432
puic                                  0.000812
not-logged-in-ca0e5c102eb9a9d65ed3    4.945489
not-logged-in-f3a5d106626fa9edf01c    0.001015
not-logged-in-f3a5d106626fa9edf01c    0.141376
BottledFarts                          0.017033
not-logged-in-588258d3c51cb2a67edd    0.001944
tkuhnle                               0.001470
weaverisabell

In [8]:
sb.glue('subject_id', subject_id)
sb.glue('losses', losses.to_dict())